In [1]:
import os
import re
import datetime as dt
import pandas as pd

In [2]:
desc ='01/03/2021 Morgan M. Mack eCheck receipt C712-17B0 Rental Income - January 2021 1,350.00 1,220.45'
desc2='01/11/2021 Yinong Jiang Check 14096 Owner Distribution - Owner payment for 01/2021 1,085.45 0.00'
desc3='Toho Water Authority Receipt Water'

m = re.search('(.*) ((e?Check)(receipt)?) ([A-Za-z0-9-]+) (.*)',desc3) 

#print(m.group(2))



    

In [3]:
def isNumber(txt):
    try:
        float(txt.replace(',',''))
        return True
    except:
        return False

def str2num(txt):
    return float(txt.replace(',',''))

def str2date(txt):
    return dt.datetime.strptime(txt,'%m/%d/%Y')
    
def isDate(txt):
    try:
        dt.datetime.strptime(txt,'%m/%d/%Y')
        return True
    except:
        return False
        

In [4]:
PAYEES=['Yinong Jiang','Hathaway Home Services Results Realty']

In [47]:
class Record:

    #---------------------------------------------------
    # Main logic of Income expesne
    #---------------------------------------------------
    
    def setIncomeExpense(self,num):
        if( 'Application Fee Income - Refund' in self.Description):
            self.Expense = num       
        elif( self.Type == 'Check'):
            self.Expense = num                   
        elif( ('Income' in self.Description) | ('Prepaid' in self.Description) ):
            self.Income = num
        elif( re.search('security deposit',self.Description,re.IGNORECASE) ):
            self.Income = num
        elif( re.search('receipt',self.Type,re.IGNORECASE)) :
            self.Income = num
        elif( re.search('refund',self.Description,re.IGNORECASE)) :
            self.Income = num
        elif( re.search('final bill close',self.Description,re.IGNORECASE)) :
            self.Income = num
        elif( re.search('Transfer from',self.Description,re.IGNORECASE)) :
            self.Income = num   
        elif( re.search('Bank Transfer',self.Description,re.IGNORECASE)) :
            self.Income = num               
        elif( re.search('receipt water',self.Description,re.IGNORECASE)) :
            self.Income = num 
        elif( re.search('Late Fee',self.Description,re.IGNORECASE)) :
            self.Income = num             
        else:
            self.Expense = num

    #----------------------------------------------------
    # Main logic for payee/type/reference/description
    #----------------------------------------------------

    def setFields(self,desc):
        
        m = None
        if( 'Check' in desc ):
            m = re.search('(.*) ((e?Check)?( ?receipt)?) ([A-Za-z0-9-]+) (.*)',desc)
        
        if( m != None ):
            self.Payee = m.group(1)
            self.Type = m.group(2)
            self.Reference = m.group(5)
            self.Description = m.group(6)
        else:
            self.Description = desc

    # Set Tax Type
    def setTaxType(self):
        if( re.search('Owner Distribution',self.Description,re.IGNORECASE)):
            self.TaxType = 'Owner'
        elif( re.search('Legal',self.Description)):
            self.TaxType = 'Legal'
        elif( re.search('Rental Income',self.Description,re.IGNORECASE)):
            self.TaxType = 'Rental Income'
        elif( re.search('Prepaid Rent',self.Description,re.IGNORECASE)):
            self.TaxType = 'Rental Income'            
        elif( re.search('Rental Income',self.Description,re.IGNORECASE)):
            self.TaxType = 'Rental Income'
        elif( re.search('Management Fees',self.Description,re.IGNORECASE)):
            self.TaxType = 'Management Fee'
        elif( re.search('Transfer',self.Description,re.IGNORECASE)):
            self.TaxType = 'Transfer' 
        # ---------------------------
        # utility
        elif( re.search('water',self.Payee,re.IGNORECASE)):
            self.TaxType = 'Utility'
        elif( re.search('electric',self.Payee,re.IGNORECASE)):
            self.TaxType = 'Utility'
        elif( re.search('electricity',self.Description,re.IGNORECASE)):
            self.TaxType = 'Utility'            
        elif( re.search('water',self.Description,re.IGNORECASE)):
            self.TaxType = 'Utility'            
            
        #----------------------------
        # repaires
        elif( re.search('Repaire',self.Description,re.IGNORECASE)):
            self.TaxType = 'Repair'
        elif( re.search('Elizabeth Wallas',self.Description,re.IGNORECASE)):
            self.TaxType = 'Repair'
        elif( re.search('Elizabeth Wallas',self.Payee,re.IGNORECASE)):
            self.TaxType = 'Repair'
        elif( re.search('Crain\'s Mechanical',self.Payee,re.IGNORECASE)):
            self.TaxType = 'Repair'
        elif( re.search('ACH Payment',self.Description,re.IGNORECASE)):
            self.TaxType = 'Repair'            
        elif( re.search('RV Gas Appliance',self.Payee,re.IGNORECASE)):
            self.TaxType = 'Repair'
        elif( re.search('Carpet',self.Payee,re.IGNORECASE)):
            self.TaxType = 'Repair'
        elif( re.search('Bugs B Gone',self.Payee,re.IGNORECASE)):
            self.TaxType = 'Repair'
            
        #----------------------------
        # Admin fees
        elif( re.search('Application',self.Description,re.IGNORECASE)):
            self.TaxType = 'Admin'            
        elif( re.search('Admin fee',self.Description,re.IGNORECASE)):
            self.TaxType = 'Admin'            
        elif( re.search('Late fee',self.Description,re.IGNORECASE)):
            self.TaxType = 'Admin'            
            
            
    def __init__(self,Date):
        self.Date = Date
        self.Payee =' '
        self.Type =' '
        self.TaxType = ' '
        self.Reference =' '
        self.Description =' '
        self.Income=0
        self.Expense=0
        self.Balance=0
        
    def __str__(self):
        return f'{self.Date}|{self.Payee}|{self.Type}|{self.Reference}|{self.Description}|{self.Income}|{self.Expense}|{self.Balance}'

    def tostr(self):
        return f'{self.Date}|{self.Payee}|{self.Type}|{self.TaxType}|{self.Reference}|{self.Description}|{self.Income}|{self.Expense}|{self.Balance}'
                
#-------------------------------------------------------------
# Test code
r = Record(dt.datetime.strptime('1/1/2021','%m/%d/%Y'))

In [6]:
def readBeginingBalance(fin,fout):
    # wait for 'Beginning'
    line = fin.readline()
    while( not line.startswith('Beginning')):
        line = fin.readline()
    m = re.search('(Beginning.*)(\d+/\d+/\d+) (-?\d+.\d{2})',line)

    r = Record(dt.datetime.strptime(m.group(2),'%m/%d/%Y'))
    r.Balance = m.group(3)
    
    fout.write(r.tostr() + '\n')
    #print(r.tostr())


In [7]:
def readOneRecord(fin,fout):
    line = ''
    p = re.compile('.* -?[0-9,]+[.]\d{2}$')
    while(True):
        readline = fin.readline()
        if( not readline ):
            return False
        line = line + readline.replace('\n','')
        if( p.match(line) != None): 
            break
        line = line + ' '
    
    # now got full line
    line.replace('\n','')
    #print(line)
    m = re.search('^(\d+/\d+/\d+) (.*) (-?[0-9,]+.\d{2}) (-?[0-9,]+.\d{2})',line)
    #print(m)
    #print(line)
    if( m ):
        r = Record(str2date(m.group(1)))
        r.setFields(m.group(2))
        r.setIncomeExpense(str2num(m.group(3)))
        r.Balance = str2num(m.group(4))
    
        r.setTaxType()
        
        fout.write(r.tostr() + '\n')

    return True



In [8]:
def createData(year,house):

    DIR=f'E:\\Documents\\{year} tax'

    file_name_in = f'{DIR}\\{house}_in.txt'
    file_name_out= f'{DIR}\\{house}_out.txt'
    
    fin = open(file_name_in,'r')
    fout = open(file_name_out,'w')


    start = True

    fout.write('Date|Payee|Type|TaxType|Reference|Descrition|Income|Expense|Balance\n')
    readBeginingBalance(fin,fout)
    while(readOneRecord(fin,fout)):
        continue;

    fin.close()
    fout.close()
    return file_name_out


In [54]:
# convert to numbers
def space2nan(v):
    if(v ==' ' ):
        return 0.0
    if( type(v) == float):
        return v
    if ( type(v) == str):
        return float(v.replace(',',''))

def readDataFrame(year,house):
    DIR=f'E:\\Documents\\{year} tax'


    # 1. Read data file
    file_name= f'{DIR}\\{house}_out.txt'

    df = pd.read_csv(file_name,delimiter='|',thousands=',')

    # 2. data cleaning

    df.Income = df.Income.apply(space2nan)
    df.Expense = df.Expense.apply(space2nan)

    # 3. Create calculated balance
    df.at[0,'C_Balance'] = df.at[0,'Balance']

    i = 1
    while i < df.C_Balance.size:
        df.at[i,'C_Balance'] = df.at[i-1,'C_Balance'] + df.at[i,'Income'] - df.at[i,'Expense'] 
        i = i+1

    return df



In [52]:
#Step 1 create data


print('Created following data files:\n')

print(createData('2022','5311'))
print(createData('2022','5510'))
print(createData('2022','5212'))

Created following data files:

E:\Documents\2022 tax\5311_out.txt
E:\Documents\2022 tax\5510_out.txt
E:\Documents\2022 tax\5212_out.txt


In [55]:
# Step 2 Validate calculated balance 

print(f'Now:{dt.datetime.now()}')

file_name = createData('2022','5510')

df = readDataFrame('2022','5510')

df

Now:2023-02-26 14:46:46.873431


,Date,Payee,Type,TaxType,Reference,Descrition,Income,Expense,Balance,C_Balance
0,2022-01-01 00:00:00,,,,,,0.0,0.0,0.0,0.0
1,2022-01-03 00:00:00,Morgan M. Mack eCheck,receipt,Admin,7B41-9880,Admin fee - Renewal,50.0,0.0,50.0,50.0
2,2022-01-03 00:00:00,Morgan M. Mack eCheck,receipt,Rental Income,7B41-9880,Rental Income - January 2022,1350.0,0.0,1400.0,1400.0
3,2022-01-10 00:00:00,Berkshire Hathaway HomeServices Results Realty,Check,Management Fee,15487,Management Fees - Management Fees for 01/2022,0.0,135.0,1265.0,1265.0
4,2022-01-10 00:00:00,Berkshire Hathaway HomeServices Results Realty,Check,Admin,15487,Admin fee - Admin fee for 01/2022,0.0,50.0,1215.0,1215.0
5,2022-01-10 00:00:00,Yinong Jiang,Check,Owner,15509,Owner Distribution - Owner payment for 01/2022,0.0,1215.0,0.0,0.0
6,2022-02-05 00:00:00,,,Rental Income,,PHYLLIS A. HORTON CC receipt 6761-EB30 Rental ...,1350.0,0.0,1350.0,1350.0
7,2022-02-05 00:00:00,,,Admin,,PHYLLIS A. HORTON CC receipt 6761-EB30 Late Fe...,135.0,0.0,1485.0,1485.0
8,2022-02-10 00:00:00,Berkshire Hathaway HomeServices Results Realty,Check,Management Fee,15600,Management Fees - Management Fees for 02/2022,0.0,135.0,1350.0,1350.0
9,2022-02-10 00:00:00,Berkshire Hathaway HomeServices Results Realty,Check,Admin,15600,Late Fee - Late Fee for 02/2022,0.0,135.0,1215.0,1215.0


In [56]:
# Step 3 House Summary

YEAR= '2022'
HOUSE = '5510'  #5212/5311/5510

df = readDataFrame(YEAR, HOUSE)

print('House: ' + HOUSE) 
df.groupby('TaxType')[['Income','Expense']].sum()


df.pivot_table(index='TaxType',aggfunc='sum',margins=True,margins_name = 'Grand Total',values=['Income','Expense'])

House: 5510


,Expense,Income
TaxType,,
,700.0,0.0
Admin,185.0,185.0
Legal,40.0,0.0
Management Fee,2142.0,0.0
Owner,12623.5,0.0
Rental Income,0.0,14670.0
Repair,1494.5,0.0
Transfer,0.0,2480.0
Utility,150.0,0.0
